### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.tools import tool
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from typing import List
from langchain.agents import AgentExecutor, create_tool_calling_agent

### 1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축

In [4]:
# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

embeddings_model = OllamaEmbeddings(model="bge-m3:latest")

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

cafe_db.save_local("../db/cafe_db")

cafe_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 4},
)

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '아메리카노',
              'menu_number': 1,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '1. 아메리카노\n'
                 '   • 가격: ₩4,500\n'
                 '   • 주요 원료: 에스프레소, 뜨거운 물\n'
                 '   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 '
                 '잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '카페라떼',
              'menu_number': 2,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '2. 카페라떼\n'
                 '   • 가격: ₩5,500\n'
                 '   • 주요 원료: 에스프레소, 스팀 밀크\n'
                 '   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 '
    

C:\Users\user\AppData\Local\Temp\ipykernel_11376\834239392.py:47: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model = OllamaEmbeddings(model="bge-m3:latest")


### 2. 3개의 도구를 정의하고 LLM에 바인딩

In [5]:
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [6]:
@tool
def wiki_summary(query: str) -> str:
    """Search Wikipedia documents based on user input (query) and return k documents."""
    wiki = WikipediaAPIWrapper(lang="ko")
    result = wiki.run(query)
    if result:
        return result
    return "관련 위키피디아 정보를 찾을 수 없습니다."

In [7]:
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 커피 메뉴 정보를 찾을 수 없습니다.")]

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant providing cafe menu information and general coffee-related knowledge.
        For information about the cafe's menu, use the db_search_menu_func tool.
        For other general information, use the wiki_summary tool.
        If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.
        """),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o")
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0.3,
#     api_key=OPENAI_API_KEY
# )

tools = [tavily_search_func, wiki_summary, db_search_menu_func]
llm_with_tools = llm.bind_tools(tools)

llm_chain = prompt | llm_with_tools

### 3. 간단한 도구 호출 체인 구현체인 구조

In [22]:
@chain
def cafe_tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("%"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        
    print("tool_msgs: \n") 
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

### 4. 테스트 질문 처리

In [24]:
# 테스트 예시
query = "아메리카노의 가격과 특징은 무엇인가요?"
response = cafe_tool_chain.invoke(query)
print(response.content)

('db_search_menu_func: \n'
 "{'name': 'db_search_menu_func', 'args': {'query': '아메리카노 가격'}, 'id': "
 "'call_M38CqwYJ29DrmlAD4Bbk8HPW', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('wiki_summary: \n'
 "{'name': 'wiki_summary', 'args': {'query': '아메리카노 특징'}, 'id': "
 "'call_XJK6cjjZxM0qaXmLNZj8S4cw', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_menu_func'
'wiki_summary'
----------------------------------------------------------------------------------------------------
아메리카노의 가격은 4,500원입니다.

#### 아메리카노의 특징:
아메리카노는 에스프레소를 뜨거운 물로 희석하여 만드는 커피 음료 중 하나입니다. 드립 커피와 비슷하지만, 풍미는 다릅니다. 에스프레소의 '샷' 수와 더해지는 물의 양에 따라 농도가 달라집니다. 이탈리아어로 "caffè americano"를 번역하면 "American coffee"이지만, 영어권에서는 이 이탈리아어 표현 그대로 사용되며, '넓은' 미국 스타일의 커피라는 의미를 내포합니다. 아메리카노는 진한 커피의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다.


### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

### 1. Few-shot 예제를 포함한 프롬프트 템플릿 작성

In [15]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("카푸치노의 가격과 특징, 유래에 대해 알려주세요.", name="example_user"),
    AIMessage("커피 정보를 검색하고, 위키피디아에서 추가 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("카푸치노: 가격 ₩5,000, 에스프레소, 스팀 밀크, 우유 거품 사용, 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.", tool_call_id="1"),    
    AIMessage("카푸치노의 가격은 ₩5,000이며, 에스프레소, 스팀 밀크, 우유 거품가 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "카푸치노", "k": 1}, "id": "2"}]),
    ToolMessage("카푸치노(이탈리아어: cappuccino, 문화어: 까뿌치노)는 에스프레소 기반의 커피 음료로, 전통적으로 스팀 우유와 함께 제조된다. 우유 대신 크림을 사용하거나 위에 계피 가루를 뿌려 먹기도 한다. 카페 라테에 비해 우유의 양이 적고 거품의 양이 많다.", tool_call_id="2"),
    AIMessage("카푸치노의 특징에 대해 알아보았습니다. 이제 카푸치노의 유래를 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "카푸치노의 유래"}, "id": "3"}]),
    ToolMessage("카푸치노(Capuccino)는 우유를 섞은 커피에 계핏가루를 뿌린 이탈리아식 커피를 말합니다. 카푸치노의 어원은 후드 (Hood)란 뜻을 지닌 이탈리아어 'cappucio'에서 비롯됐습니다. \
                이탈리아에서는 후드가 달린 갈색의 수도복을 입은 수도사를 종종 볼 수 있는데, 그들이 갈색 수도복 위에 하얀 허리끈 등을 두른 것이 마치 커피와 우유가 섞인 카푸치노의 모습과 비슷하다 하여 카푸치노의 어원이 됐습니다.", tool_call_id="3"),
    AIMessage("카푸치노(₩5,000)는 에스프레소, 스팀 밀크, 우유 거품가 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. \
                카푸치노의 어원은 후드(Hood)란 뜻을 지닌 이탈리아어 'cappucio'에서 비롯됐습니다. \
                이탈리아에의 수도사가 입는 후드가 달린 갈색의 수도복 위에 하얀 허리끈을 두른 것이 마치 커피와 우유가 섞인 카푸치노의 모습과 비슷하다 하여 카푸치노의 어원이 됐습니다.", name="example_assistant"),
]

### 2. 각 도구의 용도를 명확히 구분하는 시스템 메시지 작성

In [18]:
system = """You are an AI assistant providing cafe menu information and general food/drink-related knowledge.
For information about the cafe's menu, use the db_search_menu_func tool.
For other general information, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.
For Complex question, use multiple tools sequentially.
You should answer information by clearly distinguishing the source of information.
"""

### 3. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현

In [30]:
few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0.3,
#     api_key=OPENAI_API_KEY
# )

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [31]:
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [32]:
# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

# 응답 출력 
pprint(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '카페라떼 디저트'}, 'id': 'call_XwpLRwQJlRKjvkC4hvQNTL7t', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '카페라떼의 유래'}, 'id': 'call_cF8W1R1NixHA16UFYDGGexys', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='a22f2920-1940-49bf-983d-e3aa0c4f3813', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 2, 'menu_name': '카페라떼'}, page_content='2. 카페라떼\\n   • 가격: ₩5,500\\n   • 주요 원료: 에스프레소, 스팀 밀크\\n   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.'), Document(id='0adb659a-f0ec-4a4a-aaff-21a40d00227d', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 4, 'menu_name': '바닐라 라떼'}, page_co